有 n 个城市，按从 0 到 n-1 编号。给你一个边数组 edges，其中 edges[i] = [fromi, toi, weighti] 代表 fromi 和 toi 两个城市之间的双向加权边，距离阈值是一个整数 distanceThreshold。

返回在路径距离限制为 distanceThreshold 以内可到达城市最少的城市。如果有多个这样的城市，则返回编号最大的城市。

注意，连接城市 i 和 j 的路径的距离等于沿该路径的所有边的权重之和。


输入：n = 4, edges = [[0,1,3],[1,2,1],[1,3,4],[2,3,1]], distanceThreshold = 4
输出：3
解释：城市分布图如上。
每个城市阈值距离 distanceThreshold = 4 内的邻居城市分别是：
城市 0 -> [城市 1, 城市 2] 
城市 1 -> [城市 0, 城市 2, 城市 3] 
城市 2 -> [城市 0, 城市 1, 城市 3] 
城市 3 -> [城市 1, 城市 2] 
城市 0 和 3 在阈值距离 4 以内都有 2 个邻居城市，但是我们必须返回城市 3，因为它的编号最大。

Floyd 算法作为「多源汇最短路」算法，对于本题尤其适合。

Floyd 算法基于「动态规划」，其原始三维状态定义为 dist[p][i][j]，表示「所有从点 i 到点 j，且允许经过点集 (1,...,p) 的路径」中的最短距离。

状态转移方程：

dist[p][i][j]=min(dist[p−1][i][j],dist[p−1][i][p]+dist[p−1][p][j])
dist[p−1][i][j] 代表从 i 到 j 但必然不经过点 p 的路径，dist[p−1][i][p]+dist[p−1][p][j] 代表必然经过点 p 的路径，两者中取较小值更新 dist[p][i][j]。

不难发现任意的 dist[p][X][Y] 依赖于 dist[p−1][X][Y]，可采用「滚动数组」的方式进行优化。

将 dist 声明为二维数组，dist[i][j] 代表从点 i 到点 j 的最短距离，并采取 [枚举中转点 - 枚举起点 - 枚举终点] 三层循环的方式更新 dist[i][j]。

如此一来，跑一遍 Floyd 算法便可得出任意两点的最短距离。

通过上述推导，不难发现，我们并没提及边权的正负问题，因此 Floyd 算法对边权的正负没有限制要求（可处理正负权边的图），且能利用 Floyd 算法可能够对图中负环进行判定。

In [ ]:
from typing import List


class Solution:
    def findTheCity(
        self, n: int, edges: List[List[int]], distanceThreshold: int
    ) -> int:
        def floyd(g: list[list[int]]) -> None:
            n = len(g)
            # floyd 基本流程为三层循环: [枚举中转点 - 枚举起点 - 枚举终点] => 松弛操作
            for p in range(n):
                for i in range(n):
                    for j in range(n):
                        g[i][j] = min(g[i][j], g[i][p] + g[p][j])

        g = [[float("inf")] * n for _ in range(n)]
        # 初始化邻接矩阵
        for i in range(n):
            g[i][i] = 0
        # 存图
        for a, b, c in edges:
            g[a][b] = g[b][a] = min(g[a][b], c)
        # 最短路
        floyd(g)
        # 统计答案
        ans, cnt = -1, n + 10
        for i in range(n):
            cur = sum(1 for j in range(n) if i != j and g[i][j] <= distanceThreshold)
            if cur <= cnt:
                cnt, ans = cur, i
        return ans